# Housing Crisis Prediction - Interactive Interface

This notebook provides a Jupyter-based interface for the Housing Crisis Prediction System.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, IFrame
import requests
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Option 1: Embedded HTML Interface

In [ ]:
display(IFrame(src='../frontend/index.html', width=1400, height=1200))

## Option 2: Pure Python Widgets Interface

In [ ]:
apiUrl = widgets.Text(
    value='http://localhost:8000',
    description='API URL:',
    style={'description_width': '150px'}
)

gdp = widgets.FloatText(value=25000, description='GDP (Billions):', style={'description_width': '150px'})
cpi = widgets.FloatText(value=300, description='CPI:', style={'description_width': '150px'})
unrate = widgets.FloatSlider(value=5.5, min=0, max=20, step=0.1, description='Unemployment (%):', style={'description_width': '150px'})
fedfunds = widgets.FloatSlider(value=4.5, min=0, max=10, step=0.1, description='Fed Funds Rate (%):', style={'description_width': '150px'})
mortgage = widgets.FloatSlider(value=6.5, min=2, max=10, step=0.1, description='Mortgage Rate (%):', style={'description_width': '150px'})
houst = widgets.FloatText(value=1400, description='Housing Starts:', style={'description_width': '150px'})
horizon = widgets.IntSlider(value=12, min=1, max=36, description='Horizon (months):', style={'description_width': '150px'})
crisisDetection = widgets.Checkbox(value=True, description='Crisis Detection')

predictBtn = widgets.Button(
    description='Generate Prediction',
    button_style='primary',
    icon='rocket'
)

output = widgets.Output()

def onPredictClick(b):
    with output:
        output.clear_output()
        
        payload = {
            'data': [{
                'GDP': gdp.value,
                'CPIAUCSL': cpi.value,
                'UNRATE': unrate.value,
                'FEDFUNDS': fedfunds.value,
                'MORTGAGE30US': mortgage.value,
                'HOUST': houst.value
            }],
            'horizon': horizon.value,
            'crisisDetection': crisisDetection.value
        }
        
        try:
            response = requests.post(f'{apiUrl.value}/predict', json=payload, timeout=60)
            
            if response.status_code == 200:
                result = response.json()
                
                print(f"✅ Prediction Complete!")
                print(f"Forecast Horizon: {result['horizon']} months")
                
                if crisisDetection.value:
                    crisisLevel = result.get('crisisLevel', 'N/A')
                    crisisScore = result.get('crisisScore', 0)
                    print(f"Crisis Level: {crisisLevel}")
                    print(f"Crisis Score: {crisisScore:.3f}")
                
                predictions = result['predictions']
                months = list(range(1, len(predictions) + 1))
                
                fig = go.Figure()
                fig.add_trace(go.Scatter(
                    x=months,
                    y=predictions,
                    mode='lines+markers',
                    name='Predicted Values',
                    line=dict(color='#667eea', width=3),
                    marker=dict(size=8)
                ))
                
                fig.update_layout(
                    title='Housing Price Forecast',
                    xaxis_title='Months Ahead',
                    yaxis_title='Predicted Value',
                    template='plotly_dark',
                    height=400,
                    hovermode='x unified'
                )
                
                fig.show()
                
                if result.get('recommendations'):
                    print("\n📋 Policy Recommendations:")
                    for i, rec in enumerate(result['recommendations'], 1):
                        print(f"{i}. {rec}")
                
                df = pd.DataFrame({
                    'Month': months,
                    'Prediction': predictions
                })
                print("\n📊 Detailed Results:")
                display(df)
            else:
                print(f"❌ API Error: {response.status_code}")
                print(response.text)
                
        except Exception as e:
            print(f"❌ Error: {str(e)}")

predictBtn.on_click(onPredictClick)

economicBox = widgets.VBox([
    widgets.HTML('<h3>Economic Indicators</h3>'),
    gdp, cpi, unrate, fedfunds
])

housingBox = widgets.VBox([
    widgets.HTML('<h3>Housing Indicators</h3>'),
    mortgage, houst, horizon, crisisDetection
])

configBox = widgets.VBox([
    widgets.HTML('<h3>Configuration</h3>'),
    apiUrl,
    predictBtn
])

inputsBox = widgets.HBox([economicBox, housingBox, configBox])

display(widgets.HTML('<h1>🏠 Housing Crisis Prediction</h1>'))
display(inputsBox)
display(output)

## Option 3: Direct API Testing

In [ ]:
API_URL = 'http://localhost:8000'

response = requests.get(f'{API_URL}/health')
print('API Status:', response.json())

In [ ]:
payload = {
    'data': [{
        'GDP': 25000,
        'CPIAUCSL': 300,
        'UNRATE': 5.5,
        'FEDFUNDS': 4.5,
        'MORTGAGE30US': 6.5,
        'HOUST': 1400
    }],
    'horizon': 12,
    'crisisDetection': True
}

response = requests.post(f'{API_URL}/predict', json=payload)
result = response.json()

print(f"Predictions: {result['predictions']}")
print(f"Crisis Level: {result.get('crisisLevel', 'N/A')}")
print(f"Crisis Score: {result.get('crisisScore', 0):.3f}")